In [1]:
# Finetunnig for sentiment analysis 

from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

/home/user/new1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-01 14:45:18.324986: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 14:45:18.334984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 14:45:18.347008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 14:45:18.350515: E

In [2]:
# Chossing Base Model

# model = 'distilbert-base-uncased' # smaller and faster BERT

id2label = {0 : "Negative", 1 : "Positive"}
label2id = {"Negative" : 0, "Positive" : 1}

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2, id2label = id2label, label2id = label2id)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Loading the dataset for fine tuning

dataset = load_dataset("shawhin/imdb-truncated")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [4]:
# preprocess data

# create tokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', add_prefix_space = True)

def tokenize_function(examples):
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(text, return_tensors = "np", truncation = True, max_length = 512)
    return tokenized_inputs

# adding pad token if not exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

#tokenize training and validation dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 1000/1000 [00:00<00:00, 9297.06 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [5]:
# creating data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation matrix
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis = 1)

    return {"accuracy" : accuracy.compute(predictions=predictions, references=labels)}


In [6]:
# Performance before Fine tuning

text_list = ["It was good.", "Not a fan, don't recommend.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions : ")

for text in text_list:
    input = tokenizer.encode(text, return_tensors="pt")
    logits = model(input).logits
    predictions = torch.argmax(logits)

    print(text + "-" + id2label[predictions.tolist()])

Untrained model predictions : 
It was good.-Positive
Not a fan, don't recommend.-Positive
Better than the first one.-Positive
This is not worth watching even once.-Positive
This one is a pass.-Positive


In [7]:
# fine tuning with LoRA

peft_config = LoraConfig(task_type="SEQ_CLS", # sequence classification 
                         r=4, # intrinsic rank of trainable weight matrix
                         lora_alpha=32, # this is like a learning rate
                         lora_dropout=0.01, # probability of dropout
                         target_modules=['q_lin']) # we apply lora to query layer


model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [8]:
#hyperparameters
lr = 1e-3 # size of optimization step
batch_size = 4 # number of examples processed per optimization step
num_epoches = 10 # number of times model runs through training data

#define training arguments
training_args = TrainingArguments(
    output_dir='distilbert-base-uncased' + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epoches,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/home/user/new1/.venv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
# Trainer object to train the model

trainer = Trainer(model=model, # peft model
                  args=training_args, # hyperparameter
                  train_dataset=tokenized_dataset["train"], # training data
                  eval_dataset=tokenized_dataset["validation"], # validation data
                  tokenizer=tokenizer, # tokenizer
                  data_collator=data_collator, # dynamically pad examples in each batch
                  compute_metrics=compute_metrics,) # evaluates model

trainer.train() 

 10%|█         | 250/2500 [02:38<26:24,  1.42it/s]Trainer is attempting to log a value of "{'accuracy': 0.879}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                                  
 10%|█         | 250/2500 [04:04<26:24,  1.42it/s]

{'eval_loss': 0.4187316298484802, 'eval_accuracy': {'accuracy': 0.879}, 'eval_runtime': 86.57, 'eval_samples_per_second': 11.551, 'eval_steps_per_second': 2.888, 'epoch': 1.0}


 20%|██        | 500/2500 [06:48<18:25,  1.81it/s]   

{'loss': 0.4281, 'grad_norm': 2.698070526123047, 'learning_rate': 0.0008, 'epoch': 2.0}


Trainer is attempting to log a value of "{'accuracy': 0.87}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                                  
 20%|██        | 500/2500 [08:21<18:25,  1.81it/s]

...

Trainer is attempting to log a value of "{'accuracy': 0.884}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                                   
100%|██████████| 2500/2500 [42:39<00:00,  1.90it/s]

{'eval_loss': 0.9991391897201538, 'eval_accuracy': {'accuracy': 0.884}, 'eval_runtime': 90.7632, 'eval_samples_per_second': 11.018, 'eval_steps_per_second': 2.754, 'epoch': 10.0}


100%|██████████| 2500/2500 [42:40<00:00,  1.02s/it]

{'train_runtime': 2560.4132, 'train_samples_per_second': 3.906, 'train_steps_per_second': 0.976, 'train_loss': 0.14040502071380614, 'epoch': 10.0}


TrainOutput(global_step=2500, training_loss=0.14040502071380614, metrics={'train_runtime': 2560.4132, 'train_samples_per_second': 3.906, 'train_steps_per_second': 0.976, 'total_flos': 1112883852759936.0, 'train_loss': 0.14040502071380614, 'epoch': 10.0})

In [13]:
# Performance After Fine tuning

print("Trained model predictions : ")

for text in text_list:
    input = tokenizer.encode(text, return_tensors="pt").to("cuda")
    logits = model(input).logits
    predictions = torch.max(logits,1).indices

    print(text + "-" + id2label[predictions.tolist()[0]])

Trained model predictions : 
It was good.-Positive
Not a fan, don't recommend.-Negative
Better than the first one.-Positive
This is not worth watching even once.-Positive
This one is a pass.-Positive
